# Combining & Exporting Datasets

In [1]:
# import libraries

import pandas as pd
import numpy as np
import os

## Orders_Products_Prior

### Import files

In [2]:
# import the orders_checked.csv file and compress the numerics values

path = r'/Users/patel/Documents/CF-Data Anaylst Course/Achievement-4/Instacart Basket Analysis-Oct 2023/02 Data/'
dtypes = {'number_of_orders': 'int8', 'orders_days_of_week': 'int8', 'order_hour_of_day': 'int8','days_since_prior_order': 'float32' }

df_ords = pd.read_csv(os.path.join(path, 'Prepared Data/orders_checked.csv'), dtype=dtypes, index_col = 0)
df_ords.head()

,order_id,user_id,number_of_orders,orders_days_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [3]:
# convert the ids into string object
df_ords['order_id'] = df_ords['order_id'].astype('str')
df_ords['user_id'] = df_ords['user_id'].astype('str')

df_ords.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   number_of_orders        int8   
 3   orders_days_of_week     int8   
 4   order_hour_of_day       int8   
 5   days_since_prior_order  float32
dtypes: float32(1), int8(3), object(2)
memory usage: 101.1+ MB


In [4]:
df_ords.shape

(3421083, 6)

In [5]:
# import orders_products_prior.csv and compress numerics values
dtypes = {'add_to_cart_order' : 'int32', 'reordered': 'int8'}
df_ords_prior = pd.read_csv(os.path.join(path, 'Orginal Data/4.3_orders_products/orders_products_prior.csv'), dtype=dtypes, index_col=False)

df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
#convert numerics values
df_ords_prior['order_id'] = df_ords_prior['order_id'].astype('str')
df_ords_prior['product_id'] = df_ords_prior['product_id'].astype('str')

df_ords_prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   order_id           object
 1   product_id         object
 2   add_to_cart_order  int32 
 3   reordered          int8  
dtypes: int32(1), int8(1), object(2)
memory usage: 649.6+ MB


In [7]:
# descriptive staticts of numerics values
df_ords_prior.describe()

,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07
mean,8.351076e+00,5.896975e-01
std,7.126671e+00,4.918886e-01
min,1.000000e+00,0.000000e+00
25%,3.000000e+00,0.000000e+00
50%,6.000000e+00,1.000000e+00
75%,1.100000e+01,1.000000e+00
max,1.450000e+02,1.000000e+00


 add_to_cart_order - skewed to the right <br>
 &emsp; min = 1 <br>
 &emsp; max = 145 <br>
 &emsp; mean = 8.35 <br>
 &emsp; 25% = 3 <br>
 &emsp; 50% = 6 <br>
 &emsp; 75% = 11 <br>
 
reordered <br>
 &emsp; min = 25% = 0 <br>
 &emsp; max = 50% = 75% = 1

In [8]:
# dimensions of dataframe
df_ords_prior.shape

(32434489, 4)

### Consistency checks

#### Checking for null

In [11]:
# No null values
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

#### Checking for duplicates

In [15]:
df_dups = df_ords_prior[df_ords_prior.duplicated()]
df_dups.shape

(0, 4)

In [20]:
df_dups2 = df_ords_prior[df_ords_prior.duplicated(subset=['order_id', 'product_id'], keep=False)]
df_dups2

,order_id,product_id,add_to_cart_order,reordered


##### no duplicates or null values found

#### Check value counts

In [21]:
df_ords_prior.value_counts('add_to_cart_order')

add_to_cart_order
1      3214874
2      3058126
3      2871133
4      2664106
5      2442025
        ...   
141          1
142          1
143          1
144          1
145          1
Name: count, Length: 145, dtype: int64

In [22]:
df_ords_prior.value_counts('reordered')

reordered
1    19126536
0    13307953
Name: count, dtype: int64

1. add_to_cart_order: Total of 145 values. Right-tailed distribution <br>
2. reordered : Total of 2 values (0, 1) -> 59% rows have 1 (more reorders)

## Merge orders and orders_products_prior datasets

#### Inner join used to merge datasets

In [69]:
# using inner join, merge datasets

df_ords_merge = df_ords.merge(df_ords_prior, on=['order_id'], indicator=True)

df_ords_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                object  
 1   user_id                 object  
 2   number_of_orders        int8    
 3   orders_days_of_week     int8    
 4   order_hour_of_day       int8    
 5   days_since_prior_order  float32 
 6   product_id              object  
 7   add_to_cart_order       int32   
 8   reordered               int8    
 9   _merge                  category
dtypes: category(1), float32(1), int32(1), int8(4), object(3)
memory usage: 1.1+ GB


In [79]:
# check number of rows and columns in the merged dataset
df_ords_merge.shape

(32434489, 10)

In [71]:
df_ords_merge.head()

,order_id,user_id,number_of_orders,orders_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


#### Conducting checks (on left merge) 

##### left merged dataset (df_ords_mergeleft) not used. Below code is to investigate null values

In [72]:
# merging using left join on orders with orders_products_prior
df_ords_mergeleft = df_ords.merge(df_ords_prior, on=['order_id'], how='left', indicator=True)

df_ords_mergeleft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32640698 entries, 0 to 32640697
Data columns (total 10 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                object  
 1   user_id                 object  
 2   number_of_orders        int8    
 3   orders_days_of_week     int8    
 4   order_hour_of_day       int8    
 5   days_since_prior_order  float32 
 6   product_id              object  
 7   add_to_cart_order       float64 
 8   reordered               float64 
 9   _merge                  category
dtypes: category(1), float32(1), float64(2), int8(3), object(3)
memory usage: 1.5+ GB


In [73]:
# df_ords_mergeleft = 32640698 rows
# df_ords_merge = 32434489

# rows difference = 206209

32640698 - 32434489

206209

In [74]:
# checking null values in mergeleft
df_ords_mergeleft.isnull().sum()

order_id                        0
user_id                         0
number_of_orders                0
orders_days_of_week             0
order_hour_of_day               0
days_since_prior_order    2078068
product_id                 206209
add_to_cart_order          206209
reordered                  206209
_merge                          0
dtype: int64

In [77]:
# checking on merge value for each row
df_ords_mergeleft.value_counts('_merge')

_merge
both          32434489
left_only       206209
right_only           0
Name: count, dtype: int64

In [75]:
# investigating null values.
df_ords_mergeleft[df_ords_mergeleft['product_id'].isnull()]

,order_id,user_id,number_of_orders,orders_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
59,1187899,1,11,4,8,14.0,NaN,NaN,NaN,left_only
255,1492625,2,15,1,11,30.0,NaN,NaN,NaN,left_only
344,2774568,3,13,5,15,11.0,NaN,NaN,NaN,left_only
363,329954,4,6,3,12,30.0,NaN,NaN,NaN,left_only
401,2196797,5,5,0,11,6.0,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
32639379,1716008,206205,4,1,16,10.0,NaN,NaN,NaN,left_only
32639665,1043943,206206,68,0,20,0.0,NaN,NaN,NaN,left_only
32639889,2821651,206207,17,2,13,14.0,NaN,NaN,NaN,left_only
32640567,803273,206208,50,5,11,4.0,NaN,NaN,NaN,left_only


##### every user has 1 order missing in orders_products_prior

## Exporting merged dataset

In [78]:
df_ords_merge.to_pickle(os.path.join(path, 'Prepared Data/orders_products_combined.pkl'))